# Setup

In [1]:
import numpy as np
import tensorflow as tf

# Helper Functions
- One-Hot Encoder

In [2]:
def one_hot(x):
    
    x = tf.keras.backend.argmax(x)
    x = tf.one_hot(indices=x, depth=78)
    x = tf.keras.layers.RepeatVector(n=1)(x)
    
    return x

# Data
## Overview
- `X`: (m, $T_x$, 78) dimensional array. 
    - We have m training examples, each of which is a snippet of $T_x =30$ musical values. 
    - At each time step, the input is one of 78 different possible values, represented as a one-hot vector. 
        - For example, X[i,t,:] is a one-hot vector representing the value of the i-th example at time t. 

- `Y`: $(T_y, m, 78)$ dimensional array
    - This is essentially the same as `X`, but shifted one step to the left (to the past). 
    - Notice that the data in `Y` is **reordered** to be dimension $(T_y, m, 78)$, where $T_y = T_x$. This format makes it more convenient to feed into LSTM later.
    - We're using the previous values to predict the next value.
        - So our sequence model will try to predict $y^{\langle t \rangle}$ given $x^{\langle 1\rangle}, \ldots, x^{\langle t \rangle}$. 

- `n_x`: The number of unique values in this dataset. This should be 78. 

- `idx_to_values`: python dictionary mapping integers 0 through 77 to musical values.

In [3]:
X, Y, n_x, idx_to_values = np.load('jazz_data.npy')

print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_x)
print('shape of X:', X.shape)
print('Shape of Y:', Y.shape)

number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
shape of X: (60, 30, 78)
Shape of Y: (30, 60, 78)


# Model
* $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, \cdots, x^{\langle T_x \rangle})$ is a window of size $T_x$ scanned over the musical corpus. 
* Each $x^{\langle t \rangle}$ is an index corresponding to a value.
* $\hat{y}^{t}$ is the prediction for the next value.
* We will be training the model on random snippets of 30 values taken from a much longer piece of music. 
    - Thus, we won't bother to set the first input $x^{\langle 1 \rangle} = \vec{0}$, since most of these snippets of audio start somewhere in the middle of a piece of music. 
    - We are setting each of the snippets to have the same length $T_x = 30$ to make vectorization easier.

1. Build model

In [4]:
# number of total time-steps & hidden units of LSTM cell
T_x, n_a = 30,  64

# initialize different layers
Reshapor = tf.keras.layers.Reshape((1, n_x))
LSTM = tf.keras.layers.LSTM(units=n_a, return_state=True)
Densor = tf.keras.layers.Dense(units=n_x, activation='softmax')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
def jazz_model(T_x, n_a, n_x):
    
    # Input layers: 
    X = tf.keras.layers.Input(shape=(T_x, n_x))
    
    # Initialize hidden state a0 and cell state c0
    a0 = tf.keras.layers.Input(shape=(n_a, ), name='a0')
    c0 = tf.keras.layers.Input(shape=(n_a, ), name='c0')
    a = a0
    c = c0
    
    # Initialize output
    outputs = []
    
    # Loop
    for t in range(T_x):
        
        ## select the t-th time-step vector from X
        x = tf.keras.layers.Lambda(lambda z: z[:, t, :])(X)
        
        ## reshape x to shape (1, n_x)
        x = Reshapor(x)
        
        ## one step of LSTM
        a, _, c = LSTM(inputs=x, initial_state=[a, c])
        
        ## compute output
        output = Densor(a)
        
        ## record output
        outputs.append(output)
    
    # Create model instance
    model = tf.keras.models.Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model

In [6]:
model = jazz_model(T_x, n_a, n_x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 78)]     0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 78)           0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 78)        0           lambda[0][0]                     
                                                                 lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]               

2. Compile model

In [7]:
# set Adam optimizer
optimizer = tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

# compile model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

3. Training

In [8]:
# initialize hidden state and cell state to be zero
m = X.shape[0]
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

# training
## list(Y) is to convert Y to a list with 30 items, each of shape (60, 78)
model.fit(x=[X, a0, c0], y=list(Y), epochs=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
60/60 [==============================] - 9s 155ms/sample - loss: 125.6649 - dense_loss: 4.3522 - dense_1_loss: 4.3459 - dense_2_loss: 4.3339 - dense_3_loss: 4.3364 - dense_4_loss: 4.3298 - dense_5_loss: 4.3319 - dense_6_loss: 4.3296 - dense_7_loss: 4.3262 - dense_8_loss: 4.3321 - dense_9_loss: 4.3269 - dense_10_loss: 4.3313 - dense_11_loss: 4.3330 - dense_12_loss: 4.3331 - dense_13_loss: 4.3246 - dense_14_loss: 4.3270 - dense_15_loss: 4.3288 - dense_16_loss: 4.3281 - dense_17_loss: 4.3347 - dense_18_loss: 4.3217 - dense_19_loss: 4.3321 - dense_20_loss: 4.3355 - dense_21_loss: 4.3270 - dense_22_loss: 4.3344 - dense_23_loss: 4.3325 - dense_24_loss: 4.3306 - dense_25_loss: 4.3272 - dense_26_loss: 4.3358 - dense_27_loss: 4.3291 - dense_28_loss: 4.3281 - dense_29_loss: 0.0000e+00 - dense_acc: 0.0667 - dense_1_acc: 0.0500 - dense_2_acc: 0.0833 - dense_3_acc: 0.0333 - dense_4_acc: 0.1167 

60/60 [==============================] - 0s 1ms/sample - loss: 107.4429 - dense_loss: 4.2558 - dense_1_loss: 4.1173 - dense_2_loss: 3.9396 - dense_3_loss: 3.9102 - dense_4_loss: 3.7608 - dense_5_loss: 3.8428 - dense_6_loss: 3.8195 - dense_7_loss: 3.5625 - dense_8_loss: 3.7200 - dense_9_loss: 3.4960 - dense_10_loss: 3.5873 - dense_11_loss: 3.8243 - dense_12_loss: 3.5977 - dense_13_loss: 3.4982 - dense_14_loss: 3.5981 - dense_15_loss: 3.5469 - dense_16_loss: 3.6771 - dense_17_loss: 3.6712 - dense_18_loss: 3.5343 - dense_19_loss: 3.7615 - dense_20_loss: 3.7663 - dense_21_loss: 3.6035 - dense_22_loss: 3.6490 - dense_23_loss: 3.5710 - dense_24_loss: 3.7371 - dense_25_loss: 3.4532 - dense_26_loss: 3.5180 - dense_27_loss: 3.6411 - dense_28_loss: 3.7077 - dense_29_loss: 0.0000e+00 - dense_acc: 0.0333 - dense_1_acc: 0.0833 - dense_2_acc: 0.2167 - dense_3_acc: 0.2000 - dense_4_acc: 0.2333 - dense_5_acc: 0.1500 - dense_6_acc: 0.1333 - dense_7_acc: 0.2333 - dense_8_acc: 0.1000 - dense_9_acc: 0.166

60/60 [==============================] - 0s 1ms/sample - loss: 89.5196 - dense_loss: 4.1964 - dense_1_loss: 3.9036 - dense_2_loss: 3.5659 - dense_3_loss: 3.5222 - dense_4_loss: 3.2364 - dense_5_loss: 3.2964 - dense_6_loss: 3.2594 - dense_7_loss: 2.9453 - dense_8_loss: 2.9810 - dense_9_loss: 2.8012 - dense_10_loss: 3.0006 - dense_11_loss: 3.0721 - dense_12_loss: 2.7898 - dense_13_loss: 2.8883 - dense_14_loss: 2.9226 - dense_15_loss: 2.9217 - dense_16_loss: 2.8096 - dense_17_loss: 2.9945 - dense_18_loss: 2.8110 - dense_19_loss: 3.0988 - dense_20_loss: 2.9996 - dense_21_loss: 2.8673 - dense_22_loss: 2.9991 - dense_23_loss: 2.9295 - dense_24_loss: 3.1571 - dense_25_loss: 2.6958 - dense_26_loss: 2.9188 - dense_27_loss: 2.8499 - dense_28_loss: 2.9865 - dense_29_loss: 0.0000e+00 - dense_acc: 0.0333 - dense_1_acc: 0.2000 - dense_2_acc: 0.2500 - dense_3_acc: 0.2167 - dense_4_acc: 0.3167 - dense_5_acc: 0.1500 - dense_6_acc: 0.1500 - dense_7_acc: 0.3333 - dense_8_acc: 0.1833 - dense_9_acc: 0.2333

60/60 [==============================] - 0s 809us/sample - loss: 69.2023 - dense_loss: 4.1584 - dense_1_loss: 3.7092 - dense_2_loss: 3.1711 - dense_3_loss: 2.9795 - dense_4_loss: 2.5800 - dense_5_loss: 2.4799 - dense_6_loss: 2.5015 - dense_7_loss: 2.1491 - dense_8_loss: 2.3037 - dense_9_loss: 2.2334 - dense_10_loss: 2.2797 - dense_11_loss: 2.2482 - dense_12_loss: 2.0871 - dense_13_loss: 2.1452 - dense_14_loss: 2.2071 - dense_15_loss: 2.2209 - dense_16_loss: 2.0469 - dense_17_loss: 2.2342 - dense_18_loss: 2.0888 - dense_19_loss: 2.1816 - dense_20_loss: 2.1285 - dense_21_loss: 2.0532 - dense_22_loss: 2.1306 - dense_23_loss: 2.0900 - dense_24_loss: 2.2912 - dense_25_loss: 1.9870 - dense_26_loss: 2.1421 - dense_27_loss: 2.0821 - dense_28_loss: 2.1086 - dense_29_loss: 0.0000e+00 - dense_acc: 0.0333 - dense_1_acc: 0.2167 - dense_2_acc: 0.3167 - dense_3_acc: 0.2667 - dense_4_acc: 0.3500 - dense_5_acc: 0.2667 - dense_6_acc: 0.3000 - dense_7_acc: 0.4667 - dense_8_acc: 0.3167 - dense_9_acc: 0.38

60/60 [==============================] - 0s 782us/sample - loss: 52.0989 - dense_loss: 4.1104 - dense_1_loss: 3.4473 - dense_2_loss: 2.6733 - dense_3_loss: 2.3457 - dense_4_loss: 1.9496 - dense_5_loss: 1.7726 - dense_6_loss: 1.7827 - dense_7_loss: 1.6034 - dense_8_loss: 1.6258 - dense_9_loss: 1.5476 - dense_10_loss: 1.6369 - dense_11_loss: 1.5939 - dense_12_loss: 1.4138 - dense_13_loss: 1.5497 - dense_14_loss: 1.6348 - dense_15_loss: 1.6635 - dense_16_loss: 1.4561 - dense_17_loss: 1.5647 - dense_18_loss: 1.4693 - dense_19_loss: 1.5361 - dense_20_loss: 1.5397 - dense_21_loss: 1.4988 - dense_22_loss: 1.4472 - dense_23_loss: 1.4924 - dense_24_loss: 1.6334 - dense_25_loss: 1.4428 - dense_26_loss: 1.5569 - dense_27_loss: 1.5159 - dense_28_loss: 1.5902 - dense_29_loss: 0.0000e+00 - dense_acc: 0.0667 - dense_1_acc: 0.2333 - dense_2_acc: 0.4167 - dense_3_acc: 0.3333 - dense_4_acc: 0.4333 - dense_5_acc: 0.6167 - dense_6_acc: 0.5333 - dense_7_acc: 0.6333 - dense_8_acc: 0.7167 - dense_9_acc: 0.66

...

60/60 [==============================] - 0s 828us/sample - loss: 8.0476 - dense_loss: 3.8442 - dense_1_loss: 1.3872 - dense_2_loss: 0.4560 - dense_3_loss: 0.1559 - dense_4_loss: 0.1140 - dense_5_loss: 0.0927 - dense_6_loss: 0.0690 - dense_7_loss: 0.0629 - dense_8_loss: 0.0651 - dense_9_loss: 0.0568 - dense_10_loss: 0.0597 - dense_11_loss: 0.0555 - dense_12_loss: 0.0492 - dense_13_loss: 0.0573 - dense_14_loss: 0.0583 - dense_15_loss: 0.0659 - dense_16_loss: 0.0536 - dense_17_loss: 0.0626 - dense_18_loss: 0.0708 - dense_19_loss: 0.0734 - dense_20_loss: 0.0712 - dense_21_loss: 0.0879 - dense_22_loss: 0.0874 - dense_23_loss: 0.0829 - dense_24_loss: 0.1193 - dense_25_loss: 0.1206 - dense_26_loss: 0.1637 - dense_27_loss: 0.1725 - dense_28_loss: 0.2192 - dense_29_loss: 0.0000e+00 - dense_acc: 0.1000 - dense_1_acc: 0.6167 - dense_2_acc: 0.8833 - dense_3_acc: 1.0000 - dense_4_acc: 1.0000 - dense_5_acc: 1.0000 - dense_6_acc: 1.0000 - dense_7_acc: 1.0000 - dense_8_acc: 1.0000 - dense_9_acc: 1.000

60/60 [==============================] - 0s 849us/sample - loss: 7.9120 - dense_loss: 3.8450 - dense_1_loss: 1.3481 - dense_2_loss: 0.4144 - dense_3_loss: 0.1392 - dense_4_loss: 0.1031 - dense_5_loss: 0.0827 - dense_6_loss: 0.0639 - dense_7_loss: 0.0572 - dense_8_loss: 0.0608 - dense_9_loss: 0.0537 - dense_10_loss: 0.0572 - dense_11_loss: 0.0488 - dense_12_loss: 0.0478 - dense_13_loss: 0.0518 - dense_14_loss: 0.0581 - dense_15_loss: 0.0597 - dense_16_loss: 0.0553 - dense_17_loss: 0.0672 - dense_18_loss: 0.0672 - dense_19_loss: 0.0756 - dense_20_loss: 0.0656 - dense_21_loss: 0.0858 - dense_22_loss: 0.0853 - dense_23_loss: 0.0897 - dense_24_loss: 0.1311 - dense_25_loss: 0.1279 - dense_26_loss: 0.1800 - dense_27_loss: 0.1841 - dense_28_loss: 0.2293 - dense_29_loss: 0.0000e+00 - dense_acc: 0.1000 - dense_1_acc: 0.6333 - dense_2_acc: 0.9000 - dense_3_acc: 1.0000 - dense_4_acc: 1.0000 - dense_5_acc: 1.0000 - dense_6_acc: 1.0000 - dense_7_acc: 1.0000 - dense_8_acc: 1.0000 - dense_9_acc: 1.000

60/60 [==============================] - 0s 930us/sample - loss: 7.5981 - dense_loss: 3.8221 - dense_1_loss: 1.2744 - dense_2_loss: 0.3870 - dense_3_loss: 0.1293 - dense_4_loss: 0.0908 - dense_5_loss: 0.0783 - dense_6_loss: 0.0560 - dense_7_loss: 0.0495 - dense_8_loss: 0.0595 - dense_9_loss: 0.0475 - dense_10_loss: 0.0493 - dense_11_loss: 0.0467 - dense_12_loss: 0.0397 - dense_13_loss: 0.0484 - dense_14_loss: 0.0476 - dense_15_loss: 0.0539 - dense_16_loss: 0.0465 - dense_17_loss: 0.0559 - dense_18_loss: 0.0607 - dense_19_loss: 0.0670 - dense_20_loss: 0.0643 - dense_21_loss: 0.0747 - dense_22_loss: 0.0758 - dense_23_loss: 0.0797 - dense_24_loss: 0.0995 - dense_25_loss: 0.1142 - dense_26_loss: 0.1647 - dense_27_loss: 0.1842 - dense_28_loss: 0.2463 - dense_29_loss: 0.0000e+00 - dense_acc: 0.1000 - dense_1_acc: 0.6667 - dense_2_acc: 0.9000 - dense_3_acc: 1.0000 - dense_4_acc: 1.0000 - dense_5_acc: 1.0000 - dense_6_acc: 1.0000 - dense_7_acc: 1.0000 - dense_8_acc: 1.0000 - dense_9_acc: 1.000

60/60 [==============================] - 0s 856us/sample - loss: 7.5964 - dense_loss: 3.8146 - dense_1_loss: 1.2312 - dense_2_loss: 0.3550 - dense_3_loss: 0.1186 - dense_4_loss: 0.0834 - dense_5_loss: 0.0726 - dense_6_loss: 0.0502 - dense_7_loss: 0.0478 - dense_8_loss: 0.0532 - dense_9_loss: 0.0452 - dense_10_loss: 0.0472 - dense_11_loss: 0.0403 - dense_12_loss: 0.0392 - dense_13_loss: 0.0466 - dense_14_loss: 0.0472 - dense_15_loss: 0.0496 - dense_16_loss: 0.0488 - dense_17_loss: 0.0535 - dense_18_loss: 0.0577 - dense_19_loss: 0.0675 - dense_20_loss: 0.0671 - dense_21_loss: 0.0722 - dense_22_loss: 0.0869 - dense_23_loss: 0.0928 - dense_24_loss: 0.1077 - dense_25_loss: 0.1368 - dense_26_loss: 0.1717 - dense_27_loss: 0.1936 - dense_28_loss: 0.2982 - dense_29_loss: 0.0000e+00 - dense_acc: 0.1000 - dense_1_acc: 0.6833 - dense_2_acc: 0.9167 - dense_3_acc: 1.0000 - dense_4_acc: 1.0000 - dense_5_acc: 1.0000 - dense_6_acc: 1.0000 - dense_7_acc: 1.0000 - dense_8_acc: 1.0000 - dense_9_acc: 1.000

60/60 [==============================] - 0s 799us/sample - loss: 7.2661 - dense_loss: 3.8131 - dense_1_loss: 1.1765 - dense_2_loss: 0.3378 - dense_3_loss: 0.1069 - dense_4_loss: 0.0779 - dense_5_loss: 0.0696 - dense_6_loss: 0.0494 - dense_7_loss: 0.0445 - dense_8_loss: 0.0525 - dense_9_loss: 0.0432 - dense_10_loss: 0.0463 - dense_11_loss: 0.0401 - dense_12_loss: 0.0400 - dense_13_loss: 0.0424 - dense_14_loss: 0.0453 - dense_15_loss: 0.0490 - dense_16_loss: 0.0494 - dense_17_loss: 0.0526 - dense_18_loss: 0.0503 - dense_19_loss: 0.0602 - dense_20_loss: 0.0558 - dense_21_loss: 0.0612 - dense_22_loss: 0.0706 - dense_23_loss: 0.0690 - dense_24_loss: 0.0921 - dense_25_loss: 0.1102 - dense_26_loss: 0.1472 - dense_27_loss: 0.1541 - dense_28_loss: 0.2495 - dense_29_loss: 0.0000e+00 - dense_acc: 0.1000 - dense_1_acc: 0.7000 - dense_2_acc: 0.9167 - dense_3_acc: 1.0000 - dense_4_acc: 1.0000 - dense_5_acc: 1.0000 - dense_6_acc: 1.0000 - dense_7_acc: 1.0000 - dense_8_acc: 1.0000 - dense_9_acc: 1.000

# Generate Jazz
1. Build model: generate $T_y$ time-steps of music

In [9]:
def generate_music_model(LSTM, Densor, n_x, n_a, T_y):
    
    # Input layer
    x0 = tf.keras.layers.Input(shape=(1, n_x))
    x = x0
    
    # Initialize hidden state a0 and cell state c0
    a0 = tf.keras.layers.Input(shape=(n_a, ), name='a0')
    c0 = tf.keras.layers.Input(shape=(n_a, ), name='c0')
    a = a0
    c = c0
    
    # Initialize output
    outputs = []
    
    # Loop
    for t in range(T_y):
        
        ## one step of LSTM
        a, _, c = LSTM(inputs=x, initial_state=[a, c])
        
        ## compute output
        output = Densor(a)
        
        ## record output
        outputs.append(output)
        
        ## generate next value for x
        x = tf.keras.layers.Lambda(one_hot)(output)
    
    # Create model instance
    model = tf.keras.models.Model(inputs=[x0, a0, c0], outputs=outputs)
    
    return model

In [10]:
inference_model = generate_music_model(LSTM, Densor, n_x, n_a, T_y=50)

inference_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 78)]      0                                            
__________________________________________________________________________________________________
a0 (InputLayer)                 [(None, 64)]         0                                            
__________________________________________________________________________________________________
c0 (InputLayer)                 [(None, 64)]         0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 64), (None,  36608       input_2[0][0]                    
                                                                 a0[0][0]                   

2. Predict and Sample

In [11]:
def predict_sample(inference_model, x0, a0, c0):
    
    # Predict and output a list of length = T_y, each element is an array of shape (1, n_x)
    preds = inference_model.predict(x=[x0, a0, c0])
    
    # Convert prediction to an ndarray of indices with the max probabilities, shape (T_y, 1)
    indices = np.argmax(preds, axis=-1)
    
    # Convert indices to ont-hot vectors of shape (T_y, n_x)
    results = tf.keras.utils.to_categorical(y=indices, num_classes=n_x)
    
    return results, indices

In [12]:
# initialization for x0, a0, c0
x0 = np.zeros((1, 1, n_x))
a0 = np.zeros((1, n_a))
c0 = np.zeros((1, n_a))

# predict and sample
results, indices = predict_sample(inference_model, x0, a0, c0)